### Como manipular XML com XPath

In [2]:
from gensim.models import KeyedVectors
import unicodedata

import gzip
import xml.etree.ElementTree as ET

with gzip.open('pordesc2018.xml.gz') as pordesc2018:
    tree = ET.parse(pordesc2018)

FileNotFoundError: [Errno 2] No such file or directory: 'pordesc2018.xml.gz'

### Monta Dicionario

In [ ]:
from unicodedata import normalize
def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

In [ ]:
wordModel = KeyedVectors.load_word2vec_format('english_unigram.bin', binary=True)

dictMesh = {}
for d in tree.findall("./DescriptorRecord"):
    terms = []
    
    qualifier = "#"
    
    for aql in d.findall('.AllowableQualifiersList/AllowableQualifier/QualifierReferredTo/QualifierName'):
        teste_qualifier = aql.find('./String').text
        if(teste_qualifier == 'anatomy & histology' or teste_qualifier == 'pharmacology' or teste_qualifier == 'methods' or teste_qualifier == 'diagnosis'):
            qualifier = teste_qualifier
            break
    
    for c in d.findall('.ConceptList/'):
        if c.find('./ScopeNote') != None:
            scope = c.find('./ScopeNote').text.replace('\n','').strip()
        for t in c.findall('./TermList/'):
            terms.append(t.find('./String').text)
            
            palavra_similar = []
            
            if t.find('./String').text.lower() in wordModel.vocab:
                sem_assento = remover_acentos(t.find('./String').text)
                palavra_similar = wordModel.most_similar_cosmul(sem_assento.lower(),topn=10)
                for similar, porcentagem in palavra_similar:
                    if(porcentagem > 0.9):
                        terms.append("<i>"+similar+"</i>")
                        
    #Usa isso somente porque o dicionario tem os nomes em portugues
    
    name = d.find('.DescriptorName/String').text
    name = name.split('[')
    name_new = name[1].replace(']', '')

            
    dictMesh[d.find('.DescriptorUI').text] = {
        'ID': d.find('.DescriptorUI').text,
        'name': name_new,
        'scope': scope,
        'terms': sorted(set(terms)),
        'qualifier': qualifier
    }
                
    #break
len(dictMesh)

### Salva Dicionario

In [4]:
import gzip, pickle

with gzip.open('dictMesh.dict_eng.gz','wb') as fp:
    pickle.dump(dictMesh,fp)
    fp.close()

### Carrega Dicionario

In [5]:
import gzip, pickle

with gzip.open('dictMesh.dict_eng.gz','rb') as fp:
    dictMesh = pickle.load(fp)
    fp.close()
    
dictMesh['D000068579']

{'ID': 'D000068579',
 'name': 'Celecoxib',
 'qualifier': 'pharmacology',
 'scope': 'A pyrazole derivative and selective CYCLOOXYGENASE 2 INHIBITOR that is used to treat symptoms associated with RHEUMATOID ARTHRITIS; OSTEOARTHRITIS and JUVENILE ARTHRITIS, as well as the management of ACUTE PAIN.',
 'terms': {'4-(5-(4-methylphenyl)-3-(trifluoromethyl)-1H-pyrazol-1-yl)benzenesulfonamide',
  '58635, SC',
  '<i>cozaar</i>',
  '<i>losartan</i>',
  '<i>norvasc</i>',
  '<i>zocor</i>',
  'Celebrex',
  'Celecoxib',
  'Celecoxibe',
  'SC 58635',
  'SC-58635',
  'SC58635'}}

In [6]:
palavra = 'Calcimicina'
for dui in dictMesh:
    d = dictMesh[dui]
    for t in d['terms']:
        if t.lower() == palavra.lower():
            print(t)
            print(d['scope'])

Calcimicina
An ionophorous, polyether antibiotic from Streptomyces chartreusensis. It binds and transports CALCIUM and other divalent cations across membranes and uncouples oxidative phosphorylation while inhibiting ATPase of rat liver mitochondria. The substance is used mostly as a biochemical tool to study the role of divalent cations in various biological systems.
